# Implementing binary decision trees with early stopping
[Programming assignment 3](https://www.coursera.org/learn/ml-classification/supplement/AqDoX/decision-trees-in-practice) of *Machine Learning: Classification* by University of Washington on Coursera.

In [programming 2](Implementing binary decision trees.ipynb), we have implemented a basic version of decision tree. Here, we add more early stopping criteria to prevent overfitting.

# 1. Prepare the data

In [4]:
import pandas as pd
import numpy as np
loans = pd.read_csv('../Data/lending-club-data.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Load data 

In [5]:
#  reassign the labels to have +1 for a safe loan, and -1 for a risky (bad) loan
loans['safe_loans'] = loans['bad_loans'].map({0: +1, 1: -1})
loans.drop('bad_loans', axis=1)
# consider four features
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'
# extract these columns from the dataset and discard others
loans = loans[features + [target]]

## One-hot encoding
By one-hot encoding, we have only numeric features. In this case, each encoded feature is either 1 or 0.

In [6]:
loans = pd.get_dummies(loans)
loans.head(5)
print(loans.shape)

(122607, 26)


## balance the two classes in the dataset

In [7]:
# the train and test set index
import json
train_idx_file = '../data/module-5-assignment-2-train-idx.json'
test_idx_file = '../data/module-5-assignment-2-test-idx.json'
with open(train_idx_file) as f:
    train_idx = json.load(f)
with open(test_idx_file) as f:
    test_idx = json.load(f)
train_data = loans.iloc[train_idx, :]
test_data = loans.iloc[test_idx, :]
print('train shape: ', train_data.shape, '. \nValue counts: \n', train_data['safe_loans'].value_counts())
print('test shape: ', test_data.shape, test_data['safe_loans'].value_counts())

train shape:  (37224, 26) . 
Value counts: 
  1    18748
-1    18476
Name: safe_loans, dtype: int64
test shape:  (9284, 26) -1    4674
 1    4610
Name: safe_loans, dtype: int64


In [8]:
train_data.columns.shape

(26,)

# 2。Implement a binary decision tree

## Function to count number of mistakes while predicting majority class
In each intermediate node, we label it with the majority class. Then, the misclassification rate can be calculated. This is used to determine the best feature for splitting.

**Note:** Keep in mind that in order to compute the number of mistakes for a majority classifier, we only need the label (y values) of the data points in the node.

In [9]:
def intermediate_node_num_mistakes(labels_in_node):
    num_safe_loans = np.count_nonzero(labels_in_node == 1)
    num_risky_loans = np.count_nonzero(labels_in_node == -1)
    return num_risky_loans if num_safe_loans > num_risky_loans else num_safe_loans

In [10]:
# test case 1
example_labels = np.array([-1, -1, 1, -1, -1])
if intermediate_node_num_mistakes(example_labels) == 1:
    print('Test passed!')
else:
    print('Test 1 failed... try again!')

Test passed!


In [11]:
# test case 2
example_labels = np.array([-1, -1, 1, 1, 1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print('Test passed!')
else:
    print('Test 1 failed... try again!')

Test passed!


In [12]:
example_labels = np.array([-1, -1, -1, -1, -1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print('Test passed!')
else:
    print('Test 1 failed... try again!')

Test passed!


## Function to pick best feature to split on
The function will loop through the list of possible features, and consider splitting on each of them. It will calculate the classification error of each split and return the feature that had the smallest classification error when split on.

In [13]:
def best_splitting_feature(data, features, target):
    best_feature = None
    min_mistakes = float('Inf')
    for feature in features:
        # split into two subsets: left for 0 and right for 1
        left_split = data[data[feature] == 0]
        right_split = data[data[feature] == 1]
        # number of misclassifications
        left_mistakes = intermediate_node_num_mistakes(left_split[target])
        right_mistakes = intermediate_node_num_mistakes(right_split[target])
        # error rate is: (left_mistakes + right_mistakes) / number of records in data
        # since number of records in data remains the same for this splitting, no need to compute
        mistakes = left_mistakes + right_mistakes
        if mistakes < min_mistakes:
            min_mistakes = mistakes
            best_feature = feature
    return best_feature

## Build the tree
Each node in the tree is represented as following

Early *stopping conditions* 1, 2, and 3
+ Reached a maximum depth. (set by parameter max_depth).
+ Reached a minimum node size. (set by parameter min_node_size).
+ Don't split if the gain in error reduction is too small. (set by parameter min_error_reduction).

In [14]:
class Node:
    def __init__(self):
        self.is_leaf = False
        self.predication = None
        self.left = None
        self.right = None
        self.splitting_feature = None

In [15]:
# Create a leaf node given a set of target values: majority 
def create_leaf(labels_in_node):
    leaf = Node()
    leaf.is_leaf = True
    if np.count_nonzero(labels_in_node == 1) > np.count_nonzero(labels_in_node == -1):
        leaf.predication = 1
    else:
        leaf.predication = -1
    return leaf

Recursive tree building stop criteria:
+ Condition 1: all data points in the node are from the same class
+ Condition 2: no more features available (each feature can be used for splitting once along a path in a tree)

**Early stopping criteria**
+ max depth reached
+ minimum node size reached (an intermediate node contains too few data points)
+ the classification error rate drops quite little after any splitting

In [16]:
# Early stopping condition 2: Whether one node has reached the minimum dataset size
def reached_minimum_node_size(data, min_node_size):
    return len(data) <= min_node_size

In [17]:
# early stopping condition 3: the classification error reduction is too small
def error_reduction(error_before_split, error_after_split):
    return error_before_split - error_after_split

In [40]:
def decision_tree_create(data, features, target, current_depth=0, max_depth=10, min_node_size=1, min_error_reduction=0):
    target_values = data[target];
    
    # stop
    # stopping condition1: in the same class, that is, misclassification error will be zero
    if intermediate_node_num_mistakes(target_values) == 0:
        print('Stopping condition 1 reached: all in the same class')
        return create_leaf(target_values)
    # stopping condition 2: no more features to use
    if len(features) == 0:
        print('Stopping condition 2 reached: no more features')
        return create_leaf(target_values)
    # early stopping condition 1: max depth reached
    if current_depth >= max_depth:
        print('Early stopping condition 1: max depth.')
        return create_leaf(target_values)
    # early stopping condition 2: minimum node size reached
    if reached_minimum_node_size(data, min_node_size):
        print('Early stopping condition 2: minimum node size.')
        return create_leaf(target_values)
    
    # find the best feature to split
    best_feature = best_splitting_feature(data, features, target)
    # split
    left_split = data[data[best_feature] == 0]
    right_split = data[data[best_feature] == 1]
    
    # early stopping condition 3: after we obtain the splitting, what is the error reduction?
    error_before_split = intermediate_node_num_mistakes(data[target])
    error_after_split = intermediate_node_num_mistakes(left_split[target]) + intermediate_node_num_mistakes(right_split[target])
    if error_reduction(error_before_split, error_after_split) / len(data) <= min_error_reduction:
        print('Early stopping condition 3: min error reduction.')
        return create_leaf(target_values)
    
    # remove this feature from current recursion path
    # in Python, generally do NOT change the arguments due to the reference semantics
    remaining_features = features[:]
    remaining_features.remove(best_feature)
    print('Split on feature {0} into two subsets of size {1} and {2}.'.format(best_feature, len(left_split), len(right_split)))
    
    # if the selected feature has only one value in this dataset, then either left or right_split will be empty. In this case,
    # we will build a leaf node for the empty split subset, whose class is the majority of its parent.
    if len(left_split) == 0 or len(right_split) == 0:
        print('The chosen splitting feature has only one value in the dataset.')
        return create_leaf(target_values)
    
    # recursion
    node = Node()
    node.is_leaf = False
    node.splitting_feature = best_feature
    if len(left_split) > 0:
        node.left = decision_tree_create(left_split, features, target, current_depth + 1, max_depth)
    else:
        node.left = create_leaf(target_values)
    if len(right_split) == 0:
        node.right = create_leaf(target_values)
    else:
        node.right = decision_tree_create(right_split, features, target, current_depth + 1, max_depth)
    return node

+ In the above building process, when the chosen feature has only one value in the intermediate subset D, the procedure labels such an intermediate node D as a leaf. 

+ However, a better way is to continue the tree building: for the child whose data is empty, assign it as a leaf and its class is the majority of its parent D. For the other child whose data is not empty (actually also D), continue the building process for it.

### Train the decision tree

In [59]:
# build the tree
features = list(train_data) # equivalent to my_dataframe.columns.values.tolist()
features.remove(target)
tree_new = decision_tree_create(train_data, features, target, 0, max_depth=6, min_node_size=0, min_error_reduction=0)

Split on feature term_ 36 months into two subsets of size 9223 and 28001.
Split on feature grade_A into two subsets of size 9122 and 101.
Early stopping condition 3: min error reduction.
Split on feature emp_length_n/a into two subsets of size 96 and 5.
Split on feature emp_length_< 1 year into two subsets of size 85 and 11.
Early stopping condition 3: min error reduction.
Early stopping condition 3: min error reduction.
Early stopping condition 3: min error reduction.
Split on feature grade_D into two subsets of size 23300 and 4701.
Split on feature grade_E into two subsets of size 22024 and 1276.
Split on feature grade_F into two subsets of size 21666 and 358.
Split on feature emp_length_n/a into two subsets of size 20734 and 932.
Split on feature grade_G into two subsets of size 20638 and 96.
Early stopping condition 1: max depth.
Early stopping condition 1: max depth.
Split on feature grade_A into two subsets of size 702 and 230.
Early stopping condition 1: max depth.
Early stoppin

In [58]:
# ignore the early stopping condition 2 and 3
tree_old = decision_tree_create(train_data, features, target, 0, max_depth=6, min_node_size=2000, min_error_reduction=0) 

Split on feature term_ 36 months into two subsets of size 9223 and 28001.
Split on feature grade_A into two subsets of size 9122 and 101.
Early stopping condition 3: min error reduction.
Split on feature emp_length_n/a into two subsets of size 96 and 5.
Split on feature emp_length_< 1 year into two subsets of size 85 and 11.
Early stopping condition 3: min error reduction.
Early stopping condition 3: min error reduction.
Early stopping condition 3: min error reduction.
Split on feature grade_D into two subsets of size 23300 and 4701.
Split on feature grade_E into two subsets of size 22024 and 1276.
Split on feature grade_F into two subsets of size 21666 and 358.
Split on feature emp_length_n/a into two subsets of size 20734 and 932.
Split on feature grade_G into two subsets of size 20638 and 96.
Early stopping condition 1: max depth.
Early stopping condition 1: max depth.
Split on feature grade_A into two subsets of size 702 and 230.
Early stopping condition 1: max depth.
Early stoppin

## Predication
Just preorder traversal of a binary tree

In [35]:
def classify(tree, x, annotate=False):
    if tree.is_leaf:
        if annotate:
            print('At leaf, predicting {}'.format(tree.predication))
        return tree.predication
    # goto the left or right subtree depending on the feature value
    split_feature_value = x[tree.splitting_feature]
    if annotate:
        print('Split on {} = {}'.format(tree.splitting_feature, split_feature_value))
    if split_feature_value == 0:
        return classify(tree.left, x, annotate)
    else:
        return classify(tree.right, x, annotate)

In [36]:
def evaluate_classification_error(tree, data):
    predications = data.apply(lambda record: classify(tree, record), axis=1) # apply to each row
    return (predications != data[target]).sum() / len(predications)

In [43]:
evaluate_classification_error(tree_new, test_data)

0.38377854373115039

In [51]:
evaluate_classification_error(tree_old, test_data)

0.38377854373115039

In [52]:
def count_leaves(tree):
    if tree.is_leaf:
        return 1
    return count_leaves(tree.left) + count_leaves(tree.right)

In [60]:
print('# leaves: ', count_leaves(tree_new), ', ', count_leaves(tree_old))

# leaves:  13 ,  13
